# 데이터셋 비교 (LGBM) - v1 버전

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers-v1/sample_submission.csv
/kaggle/input/lg-aimers-v1/train.csv
/kaggle/input/lg-aimers-v1/test.csv


In [2]:
submission = pd.read_csv('/kaggle/input/lg-aimers-v1/sample_submission.csv', index_col= 'ID')
train_v1 = pd.read_csv('/kaggle/input/lg-aimers-v1/train.csv', index_col=None)
test_v1 = pd.read_csv('/kaggle/input/lg-aimers-v1/test.csv', index_col = None)

In [3]:
# train_v1 = pd.concat([train_v1, eval_v1, validation_v1], ignore_index=True)

In [4]:
train_v1.head()

,임신 성공 여부,임신 시도 또는 마지막 임신 경과 연수,총 시술 횟수,클리닉 내 총 시술 횟수,IVF 시술 횟수,DI 시술 횟수,총 임신 횟수,IVF 임신 횟수,DI 임신 횟수,총 출산 횟수,...,난자 기증자 나이_만26-30세,난자 기증자 나이_만31-35세,난자 기증자 나이_알 수 없음,정자 기증자 나이_만20세 이하,정자 기증자 나이_만21-25세,정자 기증자 나이_만26-30세,정자 기증자 나이_만31-35세,정자 기증자 나이_만36-40세,정자 기증자 나이_만41-45세,정자 기증자 나이_알 수 없음
0,0,0,3,3,3,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,0,2,2,2,0,1,1,0,1,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,1,0,2,1,2,0,1,1,0,1,...,0,0,1,0,0,0,0,0,0,1
4,0,0,3,3,3,0,2,2,0,2,...,0,0,1,0,0,0,0,0,0,1


In [5]:
test_v1.head()

,임신 시도 또는 마지막 임신 경과 연수,총 시술 횟수,클리닉 내 총 시술 횟수,IVF 시술 횟수,DI 시술 횟수,총 임신 횟수,IVF 임신 횟수,DI 임신 횟수,총 출산 횟수,IVF 출산 횟수,...,난자 기증자 나이_만26-30세,난자 기증자 나이_만31-35세,난자 기증자 나이_알 수 없음,정자 기증자 나이_만20세 이하,정자 기증자 나이_만21-25세,정자 기증자 나이_만26-30세,정자 기증자 나이_만31-35세,정자 기증자 나이_만36-40세,정자 기증자 나이_만41-45세,정자 기증자 나이_알 수 없음
0,0,1,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0,1,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [6]:
# all_data = pd.concat([test_v1, train_v1], ignore_index = True)
# all_data = all_data.drop('임신 성공 여부', axis= 1)

In [7]:
X = train_v1.drop('임신 성공 여부', axis= 1)
X_test = test_v1

y = train_v1['임신 성공 여부'].values

## 1. 평가 함수 정의

In [8]:
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

## 2. 파라미터 최적화

In [9]:
import re
import lightgbm as lgb
from collections import Counter
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

def clean_feature_names(feature_names):
    """
    LightGBM이 허용하지 않는 특수 문자를 제거하고, 중복된 feature name을 방지하는 함수
    """
    cleaned_names = [re.sub(r'[^가-힣a-zA-Z0-9_]', '_', col) for col in feature_names]  # 특수 문자 제거
    name_counts = Counter(cleaned_names)  # feature name 개수 확인
    
    unique_names = {}
    final_names = []
    
    for name in cleaned_names:
        if name_counts[name] > 1:  # 중복된 feature 이름이 있다면
            if name not in unique_names:
                unique_names[name] = 1
            else:
                unique_names[name] += 1
            new_name = f"{name}_{unique_names[name]}"  # 숫자 추가해서 구분
        else:
            new_name = name
        final_names.append(new_name)
    
    return final_names
    
# feature 이름을 정리
cleaned_feature_names = clean_feature_names(list(X_train.columns))  

# LightGBM Dataset 생성 (feature_pre_filter=False 추가)
bayes_dtrain = lgb.Dataset(X_train, y_train, feature_name=cleaned_feature_names, params={'feature_pre_filter': False})
bayes_dvalid = lgb.Dataset(X_valid, y_valid, feature_name=cleaned_feature_names, params={'feature_pre_filter': False})

In [10]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
# TODO: 점점 줄여나가자!!!!
param_bounds = {
    'num_leaves': (30, 100),  
    'lambda_l1': (0, 1),  
    'lambda_l2': (0, 2),  
    'feature_fraction': (0.7, 1),  
    'bagging_fraction': (0.5, 0.8),  
    'min_child_samples': (5, 50),  
    'min_child_weight': (25, 50)  
}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary', # binary classification
                'learning_rate': 0.005, # 0.01~0.001
                'bagging_freq': 1, # 0 or 1
                'force_row_wise': True,
                'random_state': 1991}

In [11]:
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping

def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터 
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터:', params)    
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=2500,
                           valid_sets=bayes_dvalid,
                           callbacks=[early_stopping(stopping_rounds=200)])
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid) 
    # roc-auc 계산
    roc_auc = roc_auc_score(y_valid, preds)
    print(f'roc-auc : {roc_auc}\n')
    
    return roc_auc

In [12]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼파라미터 범위
                                 random_state=0)

In [13]:
cleaned_feature_names

['임신_시도_또는_마지막_임신_경과_연수',
 '총_시술_횟수',
 '클리닉_내_총_시술_횟수',
 'IVF_시술_횟수',
 'DI_시술_횟수',
 '총_임신_횟수',
 'IVF_임신_횟수',
 'DI_임신_횟수',
 '총_출산_횟수',
 'IVF_출산_횟수',
 'DI_출산_횟수',
 '총_생성_배아_수',
 '미세주입된_난자_수',
 '미세주입에서_생성된_배아_수',
 '이식된_배아_수',
 '미세주입_배아_이식_수',
 '저장된_배아_수',
 '미세주입_후_저장된_배아_수',
 '해동된_배아_수',
 '해동_난자_수',
 '수집된_신선_난자_수',
 '저장된_신선_난자_수',
 '혼합된_난자_수',
 '파트너_정자와_혼합된_난자_수',
 '기증자_정자와_혼합된_난자_수',
 '난자_혼합_경과일',
 '배아_이식_경과일',
 '배아_해동_경과일',
 '특정_시술_유형_IVF',
 '특정_시술_유형_ICSI',
 '특정_시술_유형_IUI',
 '특정_시술_유형_ICI',
 '특정_시술_유형_GIFT',
 '특정_시술_유형_FER',
 '특정_시술_유형_Generic_DI',
 '특정_시술_유형_IVI',
 '특정_시술_유형_BLASTOCYST',
 '특정_시술_유형_AH',
 '특정_시술_유형_Unknown',
 '복합_시술_횟수',
 '세부_조합_횟수',
 '배란_자극_여부',
 '단일_배아_이식_여부',
 '착상_전_유전_검사_사용_여부',
 '착상_전_유전_진단_사용_여부',
 '남성_주_불임_원인',
 '남성_부_불임_원인',
 '여성_주_불임_원인',
 '여성_부_불임_원인',
 '부부_주_불임_원인',
 '부부_부_불임_원인',
 '불명확_불임_원인',
 '불임_원인___난관_질환',
 '불임_원인___남성_요인',
 '불임_원인___배란_장애',
 '불임_원인___자궁경부_문제',
 '불임_원인___자궁내막증',
 '불임_원인___정자_농도',
 '불임_원인___정자_면역학적_요인',
 '불임_원인___정자_운동성',
 '불임_원인___정자_형

In [14]:
# 베이지안 최적화 수행
# TODO: init_points 10~15, n_iter 30~70
optimizer.maximize(init_points=15, n_iter=60)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 61, 'lambda_l1': 0.6027633760716439, 'lambda_l2': 1.0897663659937937, 'feature_fraction': 0.9145568099117258, 'bagging_fraction': 0.6646440511781975, 'min_child_samples': 24, 'min_child_weight': 41.147352826666406, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 53119, number of negative: 151961
[LightGBM] [Info] Total Bins 814
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 118
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259016 -> initscore=-1.051089
[LightGBM] [Info] Start training from score -1.051089
Training until validation scores don't improve for 200 rounds
Earl

In [15]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']

In [16]:
# 정수형 하이퍼파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [17]:
# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)

In [18]:
max_params

{'bagging_fraction': 0.6033451989713122,
 'feature_fraction': 0.9930153443385332,
 'lambda_l1': 0.8742286490397031,
 'lambda_l2': 1.9213708775138554,
 'min_child_samples': 45,
 'min_child_weight': 27.479329470498726,
 'num_leaves': 31,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

## 3. 모델 훈련

In [19]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

In [20]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

In [21]:
# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    print(type(X))  # DataFrame인지 확인
    
    # 훈련용 데이터, 검증용 데이터 설정
    # X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    # X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # 데이터가 DataFrame인지 확인 후 인덱싱 방식 결정
    if isinstance(X, pd.DataFrame):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    else:  # numpy.ndarray일 경우
        X_train, X_valid = X[train_idx], X[valid_idx]

    y_train, y_valid = y[train_idx], y[valid_idx]

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train, feature_name=cleaned_feature_names, params={'feature_pre_filter': False})
    dvalid = lgb.Dataset(X_valid, y_valid, feature_name=cleaned_feature_names, params={'feature_pre_filter': False})
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params,    # 최적 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=2500, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval = lgb_roc_auc,
                          callbacks=[early_stopping(stopping_rounds=200)])
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
<class 'pandas.core.frame.DataFrame'>
[LightGBM] [Info] Number of positive: 59605, number of negative: 171110
[LightGBM] [Info] Total Bins 812
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 117
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054567
[LightGBM] [Info] Start training from score -1.054567
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1391]	valid_0's binary_logloss: 0.489311	valid_0's roc_auc: 0.736317
폴드 1 roc-auc : 0.7363168214276556

######################################## 폴드 2 / 폴드 10 ########################################
<class 'pandas.core.frame.DataFrame'>
[LightGBM] [Info] Number of positive: 59606, number of negative: 171110
[LightGBM] [Info] Total Bins 819
[LightGBM] [Info] Number of data points in the train set: 230716, number of used features: 11

## 4. 최종 결과

In [22]:
print('OOF 검증 데이터 roc-auc :', roc_auc_score(y, oof_val_preds))

OOF 검증 데이터 roc-auc : 0.740340231425398


In [23]:
submission['probability'] = oof_test_preds
submission.to_csv('v1_submission.csv')